In [2]:
import requests
import pandas as pd
import time, random
from io import StringIO


BASE_URL = ("https://stats.espncricinfo.com/ci/engine/stats/index.html?"
            "agemax1=39;ageval1=age;class=2;filter=advanced;floodlit=1;floodlit=2;"
            "home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;"
            "orderby=batted_score;result=1;result=2;result=3;result=5;runsmax1=500;runsval1=runs;"
            "size=200;spanmax2=15+Aug+2025;spanmin1=01+Jan+2018;spanval1=span;spanval2=span;"
            "team=6;template=results;type=batting;view=innings")

HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                   "(KHTML, like Gecko) Chrome/115.0 Safari/537.36"),
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://stats.espncricinfo.com/"
}

def fetch_page_html(page:int) -> str:
    url = f"{BASE_URL};page={page}"   # only append page, everything else unchanged
    r = requests.get(url, headers=HEADERS, timeout=30)
    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code} on page {page}")
    return r.text

def pick_innings_table(tables:list[pd.DataFrame]) -> pd.DataFrame | None:
    """
    ESPN tables: innings table is usually index 2.
    Fallback: find a table with common batting columns.
    """
    # try the usual index first
    try:
        df = tables[2]
        if isinstance(df, pd.DataFrame) and "Player" in df.columns:
            return df
    except Exception:
        pass

    # fallback heuristic
    wanted_cols = {"Player", "Runs", "BF", "SR"}
    for t in tables:
        if isinstance(t, pd.DataFrame) and ("Player" in t.columns) and (len(wanted_cols & set(t.columns)) >= 2):
            return t
    return None

all_pages = []
pages_scraped = 0

# Big upper bound; loop auto-stops when table not found
for page in range(1, 200):
    try:
        print(f"Fetching page {page} ...", end="")
        html = fetch_page_html(page)
        tables = pd.read_html(StringIO(html))   # avoid FutureWarning
        if len(tables) == 0:
            print(" no tables -> stop.")
            break

        df = pick_innings_table(tables)
        if df is None or df.empty:
            print(" no innings table -> stop.")
            break

        # Drop repeated header rows that sometimes appear mid-table
        if "Player" in df.columns:
            df = df[df["Player"] != "Player"]

        all_pages.append(df)
        pages_scraped += 1
        print(f" ok ({df.shape[0]} rows)")

        # polite delay to reduce blocking risk
        time.sleep(random.uniform(1.5, 3.5))

    except Exception as e:
        print(f"\nStopped on page {page} due to: {e}")
        break

if not all_pages:
    raise SystemExit("No data collected. Try rerunning or increasing delay/headers.")

final_df = pd.concat(all_pages, ignore_index=True)

# Optional: reset column names if multi-level headers sneak in
final_df.columns = [str(c) for c in final_df.columns]

# Save
out_path = "espn_odi_batting_innings_team6_2018_to_2025.csv"
final_df.to_csv(out_path, index=False)
print(f"\nDone. Pages scraped: {pages_scraped}  |  Shape: {final_df.shape}  |  Saved: {out_path}")


Fetching page 1 ... ok (200 rows)
Fetching page 2 ... ok (200 rows)
Fetching page 3 ... ok (200 rows)
Fetching page 4 ... ok (200 rows)
Fetching page 5 ... ok (200 rows)
Fetching page 6 ... ok (200 rows)
Fetching page 7 ... ok (200 rows)
Fetching page 8 ... ok (52 rows)
Fetching page 9 ... no innings table -> stop.

Done. Pages scraped: 8  |  Shape: (1452, 13)  |  Saved: espn_odi_batting_innings_team6_2018_to_2025.csv


In [3]:
final_df

,Player,Runs,Mins,BF,4s,6s,SR,Inns,Unnamed: 8,Opposition,Ground,Start Date,Unnamed: 12
0,Ishan Kishan,210,169,131,24,10,160.3,1,NaN,v Bangladesh,Chattogram,10 Dec 2022,NaN
1,Shubman Gill,208,225,149,19,9,139.59,1,NaN,v New Zealand,Hyderabad,18 Jan 2023,NaN
2,V Kohli,166*,173,110,13,8,150.9,1,NaN,v Sri Lanka,Thiruvananthapuram,15 Jan 2023,NaN
3,RG Sharma,162,-,137,20,4,118.24,1,NaN,v West Indies,Brabourne,29 Oct 2018,NaN
4,V Kohli,160*,220,159,12,2,100.62,1,NaN,v South Africa,Cape Town,7 Feb 2018,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,Mohammed Shami,DNB,-,-,-,-,-,2,NaN,v Australia,Dubai (DICS),4 Mar 2025,NaN
1448,CV Varun,DNB,-,-,-,-,-,2,NaN,v Australia,Dubai (DICS),4 Mar 2025,NaN
1449,Kuldeep Yadav,DNB,-,-,-,-,-,2,NaN,v New Zealand,Dubai (DICS),9 Mar 2025,NaN
1450,Mohammed Shami,DNB,-,-,-,-,-,2,NaN,v New Zealand,Dubai (DICS),9 Mar 2025,NaN


In [4]:
display(final_df)

,Player,Runs,Mins,BF,4s,6s,SR,Inns,Unnamed: 8,Opposition,Ground,Start Date,Unnamed: 12
0,Ishan Kishan,210,169,131,24,10,160.3,1,NaN,v Bangladesh,Chattogram,10 Dec 2022,NaN
1,Shubman Gill,208,225,149,19,9,139.59,1,NaN,v New Zealand,Hyderabad,18 Jan 2023,NaN
2,V Kohli,166*,173,110,13,8,150.9,1,NaN,v Sri Lanka,Thiruvananthapuram,15 Jan 2023,NaN
3,RG Sharma,162,-,137,20,4,118.24,1,NaN,v West Indies,Brabourne,29 Oct 2018,NaN
4,V Kohli,160*,220,159,12,2,100.62,1,NaN,v South Africa,Cape Town,7 Feb 2018,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,Mohammed Shami,DNB,-,-,-,-,-,2,NaN,v Australia,Dubai (DICS),4 Mar 2025,NaN
1448,CV Varun,DNB,-,-,-,-,-,2,NaN,v Australia,Dubai (DICS),4 Mar 2025,NaN
1449,Kuldeep Yadav,DNB,-,-,-,-,-,2,NaN,v New Zealand,Dubai (DICS),9 Mar 2025,NaN
1450,Mohammed Shami,DNB,-,-,-,-,-,2,NaN,v New Zealand,Dubai (DICS),9 Mar 2025,NaN


In [24]:
final_df.to_csv("batting_new.csv",index=False)

In [7]:
# suppose your column is named 'Runs'
final_df['Runs'] = final_df['Runs'].astype(str).str.replace('*', '', regex=False)
final_df['Runs'] = pd.to_numeric(final_df['Runs'], errors='coerce')


In [8]:
final_df

,Player,Runs,Mins,BF,4s,6s,SR,Inns,Unnamed: 8,Opposition,Ground,Start Date,Unnamed: 12
0,Ishan Kishan,210.0,169,131,24,10,160.3,1,NaN,v Bangladesh,Chattogram,10 Dec 2022,NaN
1,Shubman Gill,208.0,225,149,19,9,139.59,1,NaN,v New Zealand,Hyderabad,18 Jan 2023,NaN
2,V Kohli,166.0,173,110,13,8,150.9,1,NaN,v Sri Lanka,Thiruvananthapuram,15 Jan 2023,NaN
3,RG Sharma,162.0,-,137,20,4,118.24,1,NaN,v West Indies,Brabourne,29 Oct 2018,NaN
4,V Kohli,160.0,220,159,12,2,100.62,1,NaN,v South Africa,Cape Town,7 Feb 2018,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,Mohammed Shami,NaN,-,-,-,-,-,2,NaN,v Australia,Dubai (DICS),4 Mar 2025,NaN
1448,CV Varun,NaN,-,-,-,-,-,2,NaN,v Australia,Dubai (DICS),4 Mar 2025,NaN
1449,Kuldeep Yadav,NaN,-,-,-,-,-,2,NaN,v New Zealand,Dubai (DICS),9 Mar 2025,NaN
1450,Mohammed Shami,NaN,-,-,-,-,-,2,NaN,v New Zealand,Dubai (DICS),9 Mar 2025,NaN


In [11]:
import requests
import pandas as pd
import time, random
from io import StringIO

# --- ESPN Cricinfo query for T20I Batting (Pakistan, 2018–2025) ---
t20i_url = (
    "https://stats.espncricinfo.com/ci/engine/stats/index.html?"
    "agemax1=39;ageval1=age;class=3;filter=advanced;floodlit=1;floodlit=2;"
    "home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;"
    "orderby=batted_score;result=1;result=2;result=3;result=5;runsmax1=500;runsval1=runs;"
    "size=200;spanmax2=15+Aug+2025;spanmin1=01+Jan+2018;spanval1=span;spanval2=span;"
    "team=6;template=results;type=batting;view=innings"
)

http_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/115.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}

def grab_html(pg_num: int) -> str:
    target = f"{t20i_url};page={pg_num}"
    resp = requests.get(target, headers=http_headers, timeout=30)
    if resp.status_code != 200:
        raise RuntimeError(f"Failed at page {pg_num}, status {resp.status_code}")
    return resp.text

def locate_innings_table(frame_list: list[pd.DataFrame]) -> pd.DataFrame | None:
    # Primary attempt: ESPN usually places it at index 2
    try:
        candidate = frame_list[2]
        if "Player" in candidate.columns:
            return candidate
    except Exception:
        pass
    # Secondary attempt: detect by required batting columns
    expected = {"Player", "Runs", "BF", "SR"}
    for tab in frame_list:
        if "Player" in tab.columns and len(set(tab.columns) & expected) >= 2:
            return tab
    return None

collected_frames = []
page_counter = 0

for pg in range(1, 200):   # stop when no more pages
    try:
        print(f"Scraping page {pg}...", end="")
        html_doc = grab_html(pg)
        parsed = pd.read_html(StringIO(html_doc))
        
        if not parsed:
            print(" no tables found → stopping.")
            break
        
        innings_df = locate_innings_table(parsed)
        if innings_df is None or innings_df.empty:
            print(" innings table missing → stopping.")
            break
        
        # Drop repeated headers inside the table
        innings_df = innings_df[innings_df["Player"] != "Player"]
        
        collected_frames.append(innings_df)
        page_counter += 1
        print(f" got {innings_df.shape[0]} rows.")
        
        # polite scraping delay
        time.sleep(random.uniform(1.2, 3.0))
        
    except Exception as err:
        print(f"\nStopped at page {pg} due to error: {err}")
        break

if not collected_frames:
    raise SystemExit("No innings data collected.")

merged_data = pd.concat(collected_frames, ignore_index=True)
merged_data.columns = [str(x) for x in merged_data.columns]

save_file = "pakistan_t20i_batting_2018_2025.csv"
merged_data.to_csv(save_file, index=False)

print(f"\n✅ Finished. Pages scraped: {page_counter} | Final shape: {merged_data.shape} | File saved: {save_file}")


Scraping page 1... got 200 rows.
Scraping page 2... got 200 rows.
Scraping page 3... got 95 rows.
Scraping page 4... innings table missing → stopping.

✅ Finished. Pages scraped: 3 | Final shape: (495, 13) | File saved: pakistan_t20i_batting_2018_2025.csv


In [12]:
merged_data

,Player,Runs,Mins,BF,4s,6s,SR,Inns,Unnamed: 8,Opposition,Ground,Start Date,Unnamed: 12
0,SA Yadav,117,88,55,14,6,212.72,2,NaN,v England,Nottingham,10 Jul 2022,NaN
1,DJ Hooda,104,78,57,9,6,182.45,1,NaN,v Ireland,Dublin (Malahide),28 Jun 2022,NaN
2,KL Rahul,101*,-,54,10,5,187.03,2,NaN,v England,Manchester,3 Jul 2018,NaN
3,RG Sharma,100*,-,56,11,5,178.57,2,NaN,v England,Bristol,8 Jul 2018,NaN
4,YBK Jaiswal,100,-,49,8,7,204.08,1,NaN,v Nepal,Hangzhou,3 Oct 2023,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
490,Ravi Bishnoi,DNB,-,-,-,-,-,1,NaN,v Zimbabwe,Harare,14 Jul 2024,NaN
491,TU Deshpande,DNB,-,-,-,-,-,1,NaN,v Zimbabwe,Harare,14 Jul 2024,NaN
492,Ravi Bishnoi,DNB,-,-,-,-,-,1,NaN,v South Africa,Gqeberha,10 Nov 2024,NaN
493,Avesh Khan,DNB,-,-,-,-,-,1,NaN,v South Africa,Gqeberha,10 Nov 2024,NaN


In [13]:
import requests
import pandas as pd
import time, random
from io import StringIO

#  Exact URL for ODI bowling innings data (unchanged)
odi_bowl_url = (
    "https://stats.espncricinfo.com/ci/engine/stats/index.html?"
    "class=2;home_or_away=1;home_or_away=2;home_or_away=3;"
    "result=1;result=2;result=3;"
    "spanmin1=01+Jan+2018;spanval1=span;"
    "team=6;template=results;type=bowling;view=innings"
)

http_opts = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/115.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}

def get_html(page_number: int) -> str:
    link = f"{odi_bowl_url};page={page_number}"
    response = requests.get(link, headers=http_opts, timeout=30)
    if response.status_code != 200:
        raise RuntimeError(f"HTTP {response.status_code} at page {page_number}")
    return response.text

def extract_bowling_table(df_list: list[pd.DataFrame]) -> pd.DataFrame | None:
    # ESPN bowling innings tables often at index 2
    try:
        candidate = df_list[2]
        if "Overs" in candidate.columns and "Wkts" in candidate.columns:
            return candidate
    except Exception:
        pass
    # Fallback: look for typical bowling columns
    bowling_keys = {"Player", "Overs", "Wkts", "Runs", "Econ"}
    for tbl in df_list:
        if isinstance(tbl, pd.DataFrame) and bowling_keys & set(tbl.columns):
            return tbl
    return None

frames = []
page_count = 0

for pg in range(1, 200):
    try:
        print(f"Scraping page {pg}…", end="")
        page_html = get_html(pg)
        tables = pd.read_html(StringIO(page_html))
        
        if not tables:
            print(" no tables → stopping.")
            break
        
        bowling_df = extract_bowling_table(tables)
        if bowling_df is None or bowling_df.empty:
            print(" no bowling table → stopping.")
            break
        
        # Clean duplicate header rows
        if "Player" in bowling_df.columns:
            bowling_df = bowling_df[bowling_df["Player"] != "Player"]
        
        frames.append(bowling_df)
        page_count += 1
        print(f" retrieved {bowling_df.shape[0]} rows.")
        
        time.sleep(random.uniform(1.2, 3.0))
    except Exception as error:
        print(f"\nStopped at page {pg} due to: {error}")
        break

if not frames:
    raise SystemExit("No bowling data collected.")

final_bowl_df = pd.concat(frames, ignore_index=True)
final_bowl_df.columns = [str(c) for c in final_bowl_df.columns]

output_csv = "india_odi_bowling_innings_2018_to_present.csv"
final_bowl_df.to_csv(output_csv, index=False)

print(f"\n Done. Pages scraped: {page_count} | Rows: {final_bowl_df.shape} | Saved as: {output_csv}")


Scraping page 1… retrieved 50 rows.
Scraping page 2… retrieved 50 rows.
Scraping page 3… retrieved 50 rows.
Scraping page 4… retrieved 50 rows.
Scraping page 5… retrieved 50 rows.
Scraping page 6… retrieved 50 rows.
Scraping page 7… retrieved 50 rows.
Scraping page 8… retrieved 50 rows.
Scraping page 9… retrieved 50 rows.
Scraping page 10… retrieved 50 rows.
Scraping page 11… retrieved 50 rows.
Scraping page 12… retrieved 50 rows.
Scraping page 13… retrieved 50 rows.
Scraping page 14… retrieved 50 rows.
Scraping page 15… retrieved 50 rows.
Scraping page 16… retrieved 50 rows.
Scraping page 17… retrieved 50 rows.
Scraping page 18… retrieved 50 rows.
Scraping page 19… retrieved 50 rows.
Scraping page 20… retrieved 50 rows.
Scraping page 21… retrieved 50 rows.
Scraping page 22… retrieved 50 rows.
Scraping page 23… retrieved 50 rows.
Scraping page 24… retrieved 50 rows.
Scraping page 25… retrieved 50 rows.
Scraping page 26… retrieved 50 rows.
Scraping page 27… retrieved 50 rows.
Scraping p

In [14]:
import requests
import pandas as pd
import time, random
from io import StringIO

#  Your exact URL (for all-round innings in ODI for India)
allround_url = (
    "https://stats.espncricinfo.com/ci/engine/stats/index.html?"
    "agemax1=39;ageval1=age;bowling_hand=1;bowling_hand=2;bowling_hand=3;"
    "bowling_pacespin=1;bowling_pacespin=2;bowling_pacespin=3;"
    "class=2;filter=advanced;floodlit=1;floodlit=2;"
    "home_or_away=1;home_or_away=2;home_or_away=3;"
    "innings_number=1;innings_number=2;keeper=0;"
    "orderby=batted_score;result=1;result=2;result=3;result=5;"
    "runsmax1=500;runsval1=runs;size=200;"
    "spanmax2=15+Aug+2025;spanmin1=01+Jan+2018;spanval1=span;spanval2=span;"
    "team=6;template=results;type=allround;view=innings"
)

custom_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9"
}

def fetch_page(page_num: int) -> str:
    url = f"{allround_url};page={page_num}"
    response = requests.get(url, headers=custom_headers, timeout=30)
    if response.status_code != 200:
        raise RuntimeError(f"HTTP {response.status_code} at page {page_num}")
    return response.text

def find_allround_table(frames: list[pd.DataFrame]) -> pd.DataFrame | None:
    # Common all-round columns include both batting and bowling stats
    keys = {"Player", "Runs", "SR", "Overs", "Wkts"}
    for frame in frames:
        if isinstance(frame, pd.DataFrame) and keys & set(frame.columns):
            return frame
    return None

collected = []
page_counter = 0

for pg in range(1, 200):
    try:
        print(f"Scraping page {pg} …", end="")
        html = fetch_page(pg)
        tables = pd.read_html(StringIO(html))
        if not tables:
            print(" no tables → stopping.")
            break

        df = find_allround_table(tables)
        if df is None or df.empty:
            print(" table missing → stopping.")
            break

        if "Player" in df.columns:
            df = df[df["Player"] != "Player"]  # remove header rows

        collected.append(df)
        page_counter += 1
        print(f" got {df.shape[0]} rows")

        time.sleep(random.uniform(1.2, 3.0))
    except Exception as e:
        print(f"\nStopped at page {pg} due to: {e}")
        break

if not collected:
    raise SystemExit("No all-round data collected.")

master_df = pd.concat(collected, ignore_index=True)
master_df.columns = [str(c) for c in master_df.columns]  # flatten columns

output_filename = "india_odi_allround_innings_2018_2025.csv"
master_df.to_csv(output_filename, index=False)

print(f"\nCompleted. Pages scraped: {page_counter} | Rows fetched: {master_df.shape} | Saved to: {output_filename}")


Scraping page 1 … got 200 rows
Scraping page 2 … got 200 rows
Scraping page 3 … got 200 rows
Scraping page 4 … got 200 rows
Scraping page 5 … got 200 rows
Scraping page 6 … got 200 rows
Scraping page 7 … got 200 rows
Scraping page 8 … got 200 rows
Scraping page 9 … got 200 rows
Scraping page 10 … got 200 rows
Scraping page 11 … got 200 rows
Scraping page 12 … got 200 rows
Scraping page 13 … got 200 rows
Scraping page 14 … got 30 rows
Scraping page 15 … table missing → stopping.

Completed. Pages scraped: 14 | Rows fetched: (2630, 13) | Saved to: india_odi_allround_innings_2018_2025.csv


In [15]:
allround_df = pd.merge(
    final_df,
    final_bowl_df,
    on=["Player", "Opposition", "Ground", "Start Date"],
    how="outer",  # some innings might have only batting or only bowling
    suffixes=("_bat", "_bowl")
)

In [19]:
allround_df

,Player,Runs_bat,Mins,BF,4s,6s,SR,Inns_bat,Unnamed: 8,Opposition,...,Start Date,Unnamed: 12,Overs,Mdns,Runs_bowl,Wkts,Econ,Inns_bowl,Unnamed: 7,Unnamed: 11
0,AM Rahane,11.0,16,13,0,0,84.61,1,NaN,v South Africa,...,7 Feb 2018,NaN,DNB,-,-,-,-,2.0,NaN,NaN
1,AM Rahane,34.0,81,50,3,0,68.0,2,NaN,v South Africa,...,16 Feb 2018,NaN,DNB,-,-,-,-,1.0,NaN,NaN
2,AM Rahane,NaN,-,-,-,-,-,2,NaN,v South Africa,...,4 Feb 2018,NaN,DNB,-,-,-,-,1.0,NaN,NaN
3,AM Rahane,79.0,126,86,5,2,91.86,2,NaN,v South Africa,...,1 Feb 2018,NaN,DNB,-,-,-,-,1.0,NaN,NaN
4,AM Rahane,8.0,25,18,0,0,44.44,1,NaN,v South Africa,...,13 Feb 2018,NaN,DNB,-,-,-,-,2.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,YS Chahal,NaN,-,-,-,-,-,1,NaN,v West Indies,...,22 Jul 2022,NaN,10.0,0,58,2,5.8,2.0,NaN,NaN
1448,YS Chahal,NaN,-,-,-,-,-,2,NaN,v West Indies,...,24 Jul 2022,NaN,9.0,0,69,1,7.66,1.0,NaN,NaN
1449,YS Chahal,NaN,-,-,-,-,-,1,NaN,v West Indies,...,27 Jul 2022,NaN,4.0,0,17,4,4.25,2.0,NaN,NaN
1450,YS Chahal,3.0,-,5,0,0,60.00,2,NaN,v West Indies,...,27 Oct 2018,NaN,10.0,1,56,1,5.6,1.0,NaN,NaN


In [22]:
allround_df.to_csv("india_allrounder_dataset.csv", index=False)


In [23]:
final_bowl_df

,Player,Overs,Mdns,Runs,Wkts,Econ,Inns,Unnamed: 7,Opposition,Ground,Start Date,Unnamed: 11
0,Mohammed Shami,9.5,0,57,7,5.79,2,NaN,v New Zealand,Wankhede,15 Nov 2023,NaN
1,JJ Bumrah,7.2,3,19,6,2.59,1,NaN,v England,The Oval,12 Jul 2022,NaN
2,Mohammed Siraj,7.0,1,21,6,3.0,1,NaN,v Sri Lanka,Colombo (RPS),17 Sep 2023,NaN
3,Kuldeep Yadav,10.0,0,25,6,2.5,1,NaN,v England,Nottingham,12 Jul 2018,NaN
4,YS Chahal,10.0,0,42,6,4.2,1,NaN,v Australia,Melbourne,18 Jan 2019,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1414,V Kohli,DNB,-,-,-,-,1,NaN,v New Zealand,Dubai (DICS),9 Mar 2025,NaN
1415,SS Iyer,DNB,-,-,-,-,1,NaN,v New Zealand,Dubai (DICS),9 Mar 2025,NaN
1416,KL Rahul,DNB,-,-,-,-,1,NaN,v New Zealand,Dubai (DICS),9 Mar 2025,NaN
1417,Shubman Gill,DNB,-,-,-,-,1,NaN,v New Zealand,Dubai (DICS),9 Mar 2025,NaN


In [25]:
final_bowl_df.to_csv("bowling_new.csv", index=False)